In [8]:
import random
import pandas as pd
import numpy as np

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

import missingno as msno
import seaborn as sns
import scipy as sc

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from sklearn import manifold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn import decomposition
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import (silhouette_score, adjusted_rand_score)


from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import QuantileTransformer

# Comparaison des prédictions (modèle manuel et collaboratif)


In [ ]:
# ouvrir les données 

df = pd.read_csv('fc_data.csv')
df.loc[:,'InvoiceDate'] = pd.to_datetime(df.InvoiceDate,errors='coerce')

# Ce notebook a pour objectif de comparer la pertinence des vecteurs caractéristiques utilisateurs dans chaqu'un des deux modèles (manuel et collaboratif).
On suppose que plus les prédictions engendrées par le modèle sont bonnes, plus les vecteurs caractéristiques qui le paramétrisent sont représentatifs de la réalité. 
Pour rappel, dans le cas collaboratif, les vecteurs caractéristiques émergent de l'algorithme, il est donc "censé" capturer les phénoménes sous-jacents d'une manière plus efficace que la simple intuition. Cependant, l'algorithme, dans la forme que nous avons implémenté, ne peut que détecter des phénoménes corrélés avec la nature des items consomés par l'ensemble des utilisateurs. Si les items sont semblables ou que les utilisateurs ne font pas de différence à l'achat entre les différents items, les vecteurs caractéristiques qui émergeront seront moins pertinents.
Dans le cas manuel, nous nous basons sur notre intuition pour définir des vecteurs caractéristiques. Nous avons, entre autre, définie des caractéristiques de type transactionelles ou date pas forcément corrélées avec la nature des items.
Il est donc légitime de se demander lequel de ces deux modèles est le plus efficace et si ils sont complémentaires. On compare leurs prédictions évaluées via la métrique "rank" définie dans le notebook ("implicit_cf").
L'algorithme de prédiction du modèle collaboratif est explicité dans le notebook ("implicit_cf).
En ce qui concerne l'algorithme de prédiction du modèle manuel, nous implémentons une prédiction basée sur la distance euclidienne séparant l'item prédit du barycentre des prédictions. 
Notons que nous considérons un sous espace de 500 utilisateurs et de 500 items (les plus populaires). Les résultats obtenus peuvent donc être biaisés.

In [ ]:
# Mélanger les données

df = shuffle(df)

In [ ]:
# on rajoute une lettre N (pour number) avant chaque numéro de produit
df.loc[:,'InvoiceNo'] = 'N' + df['InvoiceNo'].astype(str) 

In [ ]:
def penalt(u_i): 
    """define a new evaluation metric based on the rank of the recommandation mark"""
    
    #u_i = shuffle(u_i)
    rank_list = []
    
    for i in range (u_i.shape[0]):
        
        if (u_i.iloc[i] == 1):
            
            rank_list.append(i)  
    
    penalt = 0
    
    for i in range (len(rank_list)):
        
        penalt = penalt + rank_list[i] - np.count_nonzero(u_i[0:rank_list[i]])
        
    penalt = penalt / len(rank_list)
    
    # get penalt in %
    penalt = (penalt * 100) / (u_i.shape[0] - len(rank_list))
        
    return(penalt)

In [ ]:
def plot_value_counts(col_name,df):       
    
    values_count = pd.DataFrame(df[col_name].dropna().value_counts())
    #print (values_count.shape)
    values_count.columns = ['count']
    # convert the index column into a regular column.
    values_count[col_name] = [ str(i) for i in values_count.index ]
    # add a column with the percentage of each data point to the sum of all data points.
    values_count['percent'] = values_count['count'].div(values_count['count'].sum()).multiply(100).round(2)
    # change the order of the columns.
    values_count = values_count.reindex([col_name,'count','percent'],axis=1)
    values_count.reset_index(drop=True,inplace=True)
    return (values_count)

Nous séparons tout d'abord le jeu initial en un jeu de test et d'entrainement. Nous choisissons un certain nombre de commandes dans la base de donnée initiale et telle sorte à ce que le nombre de commandes dans le testset représente 30% du nombre de commandes totales. Nous construisons ensuite les jeux de données manuels avec caractéristiques et les jeux de données d'entrée (matrice des transactions items > utilisateurs) pour le modèle collaboratif

In [ ]:
def netoyage_items(mod_train,mod_test):
    
    # utilisateurs dans le jeux d'entrainement    
    nb_users_train = plot_value_counts('CustomerID',df=mod_train)
    print (nb_users_train.shape)
    
    # utilisateurs dans le jeux de test 
    nb_users_test = plot_value_counts('CustomerID',df=mod_test)
    print (nb_users_test.shape)
        
    # trouver les utilisateurs qui ne sont pas communs 
    mod_test = mod_test[(mod_test.loc[:,'CustomerID'].isin(mod_train.loc[:,'CustomerID']))].reset_index(drop=True)
    mod_train = mod_train[(mod_train.loc[:,'CustomerID'].isin(mod_test.loc[:,'CustomerID']))].reset_index(drop=True)
    print (mod_test.shape)
    print (mod_train.shape)
        
    return (mod_train, mod_test)

def netoyage_users(mod_train,mod_test):
    
    # utilisateurs dans le jeux d'entrainement    
    nb_users_train = plot_value_counts('StockCode',df=mod_train)
    print (nb_users_train.shape)
    
    # utilisateurs dans le jeux de test 
    nb_users_test = plot_value_counts('StockCode',df=mod_test)
    print (nb_users_test.shape)
        
    # trouver les utilisateurs qui ne sont pas communs 
    mod_test = mod_test[(mod_test.loc[:,'StockCode'].isin(mod_train.loc[:,'StockCode']))].reset_index(drop=True)
    mod_train = mod_train[(mod_train.loc[:,'StockCode'].isin(mod_test.loc[:,'StockCode']))].reset_index(drop=True)
    print (mod_test.shape)
    print (mod_train.shape)
        
    return (mod_train, mod_test)



def traintest_split(df):
    
    valeurs_cust_trans = shuffle(plot_value_counts('StockCode',df=df))
    
    
    # effectuer une copie sur laquelle on va venir travailler dans le but d'accélérer la rapidité du programme
    
    df_test = df.copy()
    df_train = df.copy()
    
            
    for i in range (valeurs_cust_trans.shape[0]):
        print(i)
        
        df_trans = df[df.loc[:,'StockCode'] == valeurs_cust_trans.iloc[i,0]]
        
        df_trans2 = plot_value_counts('InvoiceNo',df=df_trans)        
        
        # on retire 30% du jeux de données
        for j in range (int(np.around(0.3*df_trans2.shape[0])+1)):
            # on itére sur le nombre de premières transactions que l'on considère           

            # on repére la liste des transactions correspondant à la première commande
            df_trans_date = df_trans[df_trans.loc[:,'InvoiceDate'] == np.amin(df_trans.loc[:,'InvoiceDate'])]

            ind_t = df.columns.get_loc('InvoiceNo')

            # on identifie le numéro de cette commande
            transaction = df_trans_date.iloc[0,ind_t]  

            # on supprimme du trainset les transactions liées à cette commande           
            df_train = df_train[df_train.loc[:,'InvoiceNo'] != transaction] 

            # on supprimme de "df_trans" les transactions liées à cette commande
            df_trans = df_trans[df_trans.loc[:,'InvoiceNo'] != transaction]      
        
        

    # on crée le testset 
    
    df_test = df[(df.loc[:,'InvoiceNo'].isin(df_train.loc[:,'InvoiceNo'])==False)]        
    
           
    return (df_train , df_test)

# fonctions necessaires au fonctionnement de l'algorithme collaboratif

In [2]:
# first, let us define some important functions for our collaboratif algorithm


def nonzeros(m, row):
    for index in range(m.indptr[row], m.indptr[row+1]):
        yield m.indices[index], m.data[index]
        
        

def alternating_least_squares_cg(Cui, factors, regularization=0.01, iterations=15):
    users, items = Cui.shape

    # initialize factors randomly
    X = np.random.rand(users, factors) * 0.01
    Y = np.random.rand(items, factors) * 0.01

    Cui, Ciu = Cui.tocsr(), Cui.T.tocsr()

    for iteration in range(iterations):
        least_squares_cg(Cui, X, Y, regularization)
        least_squares_cg(Ciu, Y, X, regularization)

    return (X, Y)


def least_squares_cg(Cui, X, Y, regularization, cg_steps=3):
    users, factors = X.shape
    YtY = Y.T.dot(Y) + regularization * np.eye(factors)

    for u in range(users):
        # start from previous iteration
        x = X[u]

        # calculate residual r = (YtCuPu - (YtCuY.dot(Xu), without computing YtCuY
        r = -YtY.dot(x)
        for i, confidence in nonzeros(Cui, u):
            r += (confidence - (confidence - 1) * Y[i].dot(x)) * Y[i]

        p = r.copy()
        rsold = r.dot(r)

        for it in range(cg_steps):
            # calculate Ap = YtCuYp - without actually calculating YtCuY
            Ap = YtY.dot(p)
            for i, confidence in nonzeros(Cui, u):
                Ap += (confidence - 1) * Y[i].dot(p) * Y[i]

            # standard CG update
            alpha = rsold / p.dot(Ap)
            x += alpha * p
            r -= alpha * Ap
            rsnew = r.dot(r)
            p = r + (rsnew / rsold) * p
            rsold = rsnew

        X[u] = x

In [5]:
def recommend(user_id, sparse_user_item, user_vecs, item_vecs, num_items=10):
    """fonction de recommandation """

    user_interactions = sparse_user_item[user_id,:].toarray()

    user_interactions = user_interactions.reshape(-1) + 1
    user_interactions[user_interactions > 1] = 0

    rec_vector = user_vecs[user_id,:].dot(item_vecs.T).toarray()

    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    recommend_vector = user_interactions * rec_vector_scaled

    item_idx = np.argsort(recommend_vector)[::-1][:num_items]

    artists = []
    scores = []

    for idx in item_idx:
        artists.append(data.artist.loc[data.artist_id == idx].iloc[0])
        scores.append(recommend_vector[idx])

    recommendations = pd.DataFrame({'artist': artists, 'score': scores})

    return recommendations

In [4]:
def prelim_cf(df):
    """ établir la matrice de transaction utilisateurs > items"""
    
        
    df_cf = df[['InvoiceNo','CustomerID','Quantity','StockCode','UnitPrice']]
    
    ind_cust = df_cf.columns.get_loc('CustomerID')
    ind_q = df_cf.columns.get_loc('Quantity')
    ind_price = df_cf.columns.get_loc('UnitPrice')
    ind_stock = df_cf.columns.get_loc('StockCode')
    ind_numb = df_cf.columns.get_loc('InvoiceNo')

    valeurs_cust = plot_value_counts('CustomerID',df=df_cf)
    valeurs_stock = plot_value_counts('StockCode',df=df_cf)     

    # associer à chaque couple produit/consommateur un Rui

    # supprimer les colonnes inutiles 
    valeurs_cust_rui = valeurs_cust_t.drop(columns = ['count','percent']) 
    


    # supprimer les colonnes inutiles 
    valeurs_cust_rui = valeurs_cust_t.drop(columns = ['count','percent'])

    # créer un vecteur numpy 
    vect = np.zeros(valeurs_cust_t.shape[0])
    for i in range (valeurs_stock_t.shape[0]):
        valeurs_cust_rui['{}'.format(valeurs_stock_t.iloc[i,0])] = vect[:]

    valeurs_cust_pui = valeurs_cust_rui.copy()

    # on itére sur les produits 
    for i in range (valeurs_stock_t.shape[0]):
        
        df_cf1 = df_cf[(df_cf.iloc[:,ind_stock] == valeurs_stock_t.iloc[i,0])]        
        # on itére sur les clients
        for j in range (valeurs_cust_rui.shape[0]):
            df_ij = df_cf1[(df_cf1.iloc[:,ind_cust] == valeurs_cust_rui.iloc[j,0])]
            
            valeurs_cust_rui.iloc[j,i+1] = df_ij.shape[0] 
             
    # associer aux utilisateurs une valeur numérique 
    vect = np.zeros(valeurs_cust_rui.shape[0])
    for i in range (valeurs_cust_rui.shape[0]):
        vect[i] = i
    # sauvegarder un df_pandas pour retrouver le numéro correspondant
    d = {'number': valeurs_cust_rui.iloc[:,0], 'value': vect[:]}
    df_users = pd.DataFrame(data=d)

    # associer aux items une valeur numérique 
    vect = np.zeros(valeurs_stock_t.shape[0])
    for i in range (valeurs_stock_t.shape[0]):
        vect[i] = i
    # sauvegarder un df_pandas pour retrouver le numéro correspondant
    d = {'number': valeurs_stock_t.iloc[:,0], 'value': vect[:]}
    df_items = pd.DataFrame(data=d)

    # créer la matrice utilisateur 
    d = {'it0': df_users.iloc[:,1]}
    for i in range (valeurs_stock_t.shape[0]-1):

        d.update({'it{}'.format(i+1): df_users.iloc[:,1]})
        users_matrix = pd.DataFrame(data=d)      

    # créer la matrice item
    d = {'it0': df_items.iloc[:,1]}
    for i in range (valeurs_cust_rui.shape[0]-1):
        d.update({'it{}'.format(i+1): df_items.iloc[:,1]})
        items_matrix = pd.DataFrame(data=d)       

        
    # vecteur note
    rating = np.matrix(valeurs_cust_rui.iloc[:,1::])
    rating = rating.flatten().transpose()
    
    # vecteur user
    um = np.matrix(users_matrix.iloc[:,:])
    um = um.flatten().transpose()
    
    # vecteur item
    it = np.matrix(items_matrix.iloc[:,:])
    it = it.transpose().flatten().transpose()
    
    vect = np.zeros(it.shape)

    d = {'try': vect[:,0]}
    fc_matrix = pd.DataFrame(data=d)
    fc_matrix['itemID'] = it[:,0]
    fc_matrix['userID'] = um[:,0]
    fc_matrix['rating'] = rating[:,0]
    
    
    return (fc_matrix)

# fonctions necessaires au fonctionnement de l'algorithme manuel


In [ ]:
def features(df):
    """ définir les caractéristiques items """
    
    ind_date = df.columns.get_loc('InvoiceDate')
    df['DAY'] = df.iloc[:,ind_date].dt.weekday    
    df['MONTH'] = df.iloc[:,ind_date].dt.month    
    df['HOUR'] = df.iloc[:,ind_date].dt.hour
    
    
    # repérer les indices
    ind_h = df.columns.get_loc('HOUR')
    ind_m = df.columns.get_loc('MONTH')
    ind_m = df.columns.get_loc('DAY')
    ind_price = df.columns.get_loc('UnitPrice')
    ind_cust = df.columns.get_loc('StockCode')
    ind_q = df.columns.get_loc('Quantity')
    
    ind_cbis = df.columns.get_loc('Country_bis')

    valeurs_cust = plot_value_counts('StockCode',df=df)    
      
    
    valeurs_month = plot_value_counts('MONTH',df=df)
    valeurs_month.iloc[:,0] = pd.to_numeric(valeurs_month.iloc[:,0], errors='coerce').fillna(0, downcast='infer')
    
    
    
    # ---------------------------------------- Associer les caractéristiques moyennes ------------------------------------

    #définir un vecteur numpy
    vect = np.zeros(valeurs_cust.shape[0])
    dataset_clust = valeurs_cust.iloc[:,[0]]

    #ajouter les nouvelles caractéristiques
    #dataset_clust['lasttransaction'] = vect[:]
    dataset_clust['nb_unit'] = vect[:]
    dataset_clust['prix_unit'] = vect[:]
    dataset_clust['montant'] = vect[:]
    dataset_clust['pays'] = vect[:]
    dataset_clust['nb_user_diff'] = vect[:]
    dataset_clust['nb_user_id'] = vect[:]       
    dataset_clust['frequence'] = vect[:]
    
    
    # tenir compte du jour de la semaine et de l'heure des transactions par 4 nouvelles variables
    # le jour(heure) comptant le plus grand nombre de transaction (0 à 7) et la probabilité de transaction 
    # ce jour(heure) ie le ratio nombre de transaction enregistrées le jour du maximum de transaction sur le nombre total
    # de transactions
    dataset_clust['jourmax'] = vect[:]
    dataset_clust['heuremax'] = vect[:]
    dataset_clust['jourprob'] = vect[:]
    dataset_clust['heureprob'] = vect[:]

    # associer la valeur 1 lorsque le pays est uk et 0 sinon
    df['Country_bis'][df.loc[:,'Country_bis'] == 'UK'] = 1
    df['Country_bis'][df.loc[:,'Country_bis'] == 'Foreign'] = 0

    # remplir les colonnes 
    for i in range (dataset_clust.shape[0]):
        
        df_trans =  df[df.iloc[:,ind_cust] == valeurs_cust.iloc[i,0]] 
        
        dataset_clust.loc[i,'nb_unit'] = np.mean(df_trans.iloc[:,ind_q])
        
        dataset_clust.loc[i,'prix_unit'] = np.mean(df_trans.iloc[:,ind_price])
        
        dataset_clust.loc[i,'montant'] = np.mean(df_trans.iloc[:,ind_price].multiply(df_trans.iloc[:,ind_q]))
        
        dataset_clust.loc[i,'pays'] = df_trans.iloc[0,ind_cbis]
        
        dataset_clust.loc[i,'frequence'] = (np.amax(df_trans.loc[:,'InvoiceDate']) - 
                                            np.amin(df_trans.loc[:,'InvoiceDate']))/df_trans.shape[0]
        
        # convertir le temps en seconde      
        dataset_clust.loc[i,'frequence'] = dataset_clust.loc[i,'frequence'].total_seconds()
        dataset_clust.loc[i,'frequence'] = float(dataset_clust.loc[i,'frequence'])
        
        
              
        
        valeurs_day = plot_value_counts('DAY',df=df)
        valeurs_day.iloc[:,0] = pd.to_numeric(valeurs_day.iloc[:,0], errors='coerce').fillna(0, downcast='infer') 
        
        dataset_clust['jourmax'] = valeurs_day.iloc[0,0]
        dataset_clust['jourprob'] = valeurs_day.iloc[0,1]/np.sum(valeurs_day.iloc[0,:])
        
        valeurs_hour = plot_value_counts('HOUR',df=df)
        valeurs_hour.iloc[:,0] = pd.to_numeric(valeurs_hour.iloc[:,0], errors='coerce').fillna(0, downcast='infer') 
        
        dataset_clust['heuremax'] = valeurs_hour.iloc[0,0]    
        dataset_clust['heureprob'] = valeurs_hour.iloc[0,1]/np.sum(valeurs_hour.iloc[0,:])
        
        valeurs_invoice = plot_value_counts('InvoiceNo',df=df_trans)
        dataset_clust.loc[i,'nb_user_diff'] = np.mean(valeurs_invoice.iloc[:,1])
        
        valeurs_identiques = plot_value_counts('CustomerID',df=df_trans)
        dataset_clust.loc[i,'nb_user_id'] = np.mean(valeurs_identiques.iloc[:,1]) 
        
    return(dataset_clust)

In [ ]:
def algo_manu(df_train,df_test,iu_train):  
    """algorithme de prédiction manuel """
     
    # initialiser le score 
    score = 0 


    ind_item = df_train.columns.get_loc('StockCode')   
    
    valeurs_user = plot_value_counts('CustomerID',df=df_train)
    
    divid = valeurs_user.shape[0]
    print (valeurs_user.shape[0])

    for i in range (valeurs_user.shape[0]):
        print (i)
        
        # on regarde les transactions de chaque utilisateur dans le trainset
        df_trans = df_train[df_train.loc[:,'CustomerID'] == valeurs_user.iloc[i,0]]
       
        # on regarde les transactions de chaque utilisateur dans le testset
        df_trans_t = df_test[df_test.loc[:,'CustomerID'] == valeurs_user.iloc[i,0]]
        

        # visualiser l'occurence de chaque idem (poid du barycentre)
        valeurs_stock = plot_value_counts('StockCode',df=df_trans)
        
        #valeurs_stock.iloc[:,1][valeurs_stock.iloc[:,1] > 0] = 1
        
      
        
        # idem pour le testset
        valeurs_stock_t = plot_value_counts('StockCode',df=df_trans_t)
        

        #  Créer la matrice item/carac adaptée à l'utilisateur i
        Mat_ic_red = iu_train[(iu_train.loc[:,'StockCode'].isin(valeurs_stock.iloc[:,0])==True)]
        

        # Trier les items
        Mat_ic_red = Mat_ic_red.sort_values(by=['StockCode'])
        valeurs_stock = valeurs_stock.sort_values(by=['StockCode'])

        # calculer le barycentre pour l'utilisateur i

        up = np.matrix(valeurs_stock.iloc[:,1])
        
        print (valeurs_stock)
        print (Mat_ic_red)
        ic = np.matrix(Mat_ic_red.iloc[:,1::])
        scale = StandardScaler().fit(ic)
        ic =  scale.transform(ic) 
        
       
        bar = (1. / np.sum(up)) *  np.dot(up,ic) 
        print (bar)
             
        # calcul de la distance euclidienne entre le barycentre et chaqu'un des points 

        # calculer uniquement pour les éléments non déjà commandés

        Mat_ic_none = iu_train[(iu_train.loc[:,'StockCode'].isin(valeurs_stock.iloc[:,0])==False)]

        vect = np.zeros(Mat_ic_none.shape[0])
        Mat_ic_none['distance'] = vect[:]

        ind_d = Mat_ic_none.columns.get_loc('distance')

        X = np.matrix(Mat_ic_none.iloc[:,1:-1])
        X =  scale.transform(X)      
        
       
        for j in range (Mat_ic_none.shape[0]):        

            Mat_ic_none.iloc[j,ind_d]= np.sqrt(np.dot((bar[0,:] - X[j,:]),np.transpose(bar[0,:] - X[j,:]))[0,0])
            
        # considérer dans le testset uniquement les éléments non communs au trainset
        
        valeurs_stock_t = valeurs_stock_t[(valeurs_stock_t.iloc[:,0].isin(valeurs_stock.iloc[:,0])==False)]
        
        # considérer les cas sans nouvelles transactions 
        if (valeurs_stock_t.shape[0] == 0):
            
            divid = divid - 1
            
        # calculer le rank            
        if (valeurs_stock_t.shape[0] > 0):
            
            # associer la valeurs de l'occurence de l'item lorsque la transaction à lieu dans le testset et 0 sinon
            vect = np.zeros(Mat_ic_none.shape[0])
            Mat_ic_none['test'] = vect[:]
            
            # associer les valeurs des occurences lorsque la transaction à lieu
            
            for j in range (valeurs_stock_t.shape[0]):                
                
                #print (Mat_ic_none)         
                Mat_ic_none.loc[:,'test'][Mat_ic_none.loc[:,'StockCode'] == valeurs_stock_t.iloc[j,0]] = valeurs_stock_t.iloc[j,1]
                #print (Mat_ic_none)
               
            
            # trier df_trans_t et Mat_ic_none par utilisateur
            
            Mat_ic_none = Mat_ic_none.sort_values(by=['StockCode'])
            valeurs_stock_t = valeurs_stock_t.sort_values(by=['StockCode'])
            
                        
            # trier Mat_ic_none par prédiction (de la distance la moins élevée à la plus élevée)
            
            Mat_ic_none = Mat_ic_none.sort_values(by='distance', ascending=True)
           
            # associer à la liste 'test' la valeurs 1 si l'occurence est superieure à 0
            
            Mat_ic_none.loc[:,'test'][Mat_ic_none.loc[:,'test'] > 0] = 1
            
                        
            # calculate penalties
            ind_test = Mat_ic_none.columns.get_loc('test')
            penalties = penalt(u_i=Mat_ic_none.iloc[:,ind_test])
            
            score = score + penalties
            
            
            
    score = score / divid       


    
    return (score)


In [ ]:
# on ne garde que les 100 items et utilisateurs principaux

valeurs_cust = plot_value_counts('CustomerID',df=df)
print (valeurs_cust.iloc[0:10,:])
valeurs_cust = valeurs_cust.drop(valeurs_cust.index[0])

valeurs_stock = plot_value_counts('StockCode',df=df)

n_u = 500
n_p = 500

valeurs_stock_t = valeurs_stock.iloc[0:n_p,:]
valeurs_cust_t = valeurs_cust.iloc[0:n_u,:]

In [ ]:
print (df.shape)
# ne garder que les utilisateurs communs : 
df = df[(df.loc[:,'CustomerID'].isin(valeurs_cust_t.iloc[:,0]))].reset_index(drop=True)

# ne garder que les items communs
df = df[(df.loc[:,'StockCode'].isin(valeurs_stock_t.iloc[:,0]))].reset_index(drop=True)
print (df.shape)


In [ ]:
# on crée 1 jeux de test/entrainement 

train_set, test_set = traintest_split(df=df) 

# on s'assure que les items dans le trainset et testset sont identiques 

df_train, df_test = netoyage_items(mod_train=train_set,mod_test=test_set)[:]
       
# on s'assure que les users dans le trainset et testset sont identiques 

df_train, df_test = netoyage_users(mod_train=df_train,mod_test=df_test)[:]
       

#-------------------------MANUAL ----------------------------------------------

# déterminer les caractéristiques items pour le trainset   

train_citem = features(df_train)
       

# déterminer le score de prédiction 

score_manu = algo_manu(df_train=df_train,df_test=df_test,iu_train=train_citem)
       

#-------------------------COLABORATIF  ----------------------------------------------


# travail préliminaire pour le testset

test_prelim = prelim_cf(df_test)

# travail préliminaire pour le trainset  

train_prelim = prelim_cf(df_train)
        

In [ ]:
# ------------ traite les transactions identiques entre le trainset et le testset  -------------------
# associe la valeur -1 pour faciliter un traitement ulterieur


# number of users 
users_testset = plot_value_counts(col_name='userID',df=test_prelim)
users_testset.iloc[:,0] = pd.to_numeric(users_testset.iloc[:,0], errors='coerce').fillna(0, downcast='infer')


# number of items 
items_testset = plot_value_counts(col_name='itemID',df=test_prelim)
items_testset.iloc[:,0] = pd.to_numeric(items_testset.iloc[:,0], errors='coerce').fillna(0, downcast='infer')

nb_users = users_testset.shape[0]
nb_items = items_testset.shape[0]

for ind_user in range (nb_users):
    #print (ind_user)
    
    item_matrix = test_prelim[test_prelim.loc[:,'userID'] == users_testset.iloc[ind_user,0]]
    item_matrix_tr = train_prelim[train_prelim.loc[:,'userID'] == users_testset.iloc[ind_user,0]] 
    
    item_matrix = item_matrix.sort_values(by='itemID', ascending=False)
    item_matrix_tr =item_matrix_tr.sort_values(by='itemID', ascending=False)
    
    
    for i in range (item_matrix_tr.shape[0]):  
        
        if (item_matrix.iloc[i,3]>0):
            
            if (item_matrix_tr.iloc[i,3]>0):
                
                item_matrix.iloc[i,3] = -1
                        
        
        
    if (ind_user==0):
        test_prelim2 = item_matrix
        
    if (ind_user!=0):
    
        test_prelim2  = pd.concat([test_prelim2, item_matrix],axis=0)

In [ ]:
# calcul le modèle collaboratif pour les meilleurs hyperparamètres (voir implicit cf)

data = train_prelim[['userID', 'itemID', 'rating']]

#data = data_cf[['userID', 'itemID', 'rating']]

data = data.rename(columns={'userID':'user','itemID':'artist','rating':'plays'})
df = data.copy()
print (data.columns)

# Drop NaN columns
data = data.dropna()
data = data.copy()

# Create a numeric user_id and artist_id column
data['user'] = data['user'].astype("category")
data['artist'] = data['artist'].astype("category")
data['user_id'] = data['user'].cat.codes
data['artist_id'] = data['artist'].cat.codes

# The implicit library expects data as a item-user matrix so we
# create two matricies, one for fitting the model (item-user) 
# and one for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['plays'].astype(float), (data['artist_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['plays'].astype(float), (data['user_id'], data['artist_id'])))

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 60
data_conf = (sparse_item_user * alpha_val).astype('double')

# Initialize the als model and fit it using the sparse item-user matrix
reg_param = 0.1
X, Y = alternating_least_squares_cg(Cui=data_conf ,factors=10, regularization=reg_param, iterations=10)

# Get the user and item vectors from our trained model
user_vecs = sparse.csr_matrix(X)
item_vecs = sparse.csr_matrix(Y)


# number of users 
users_testset = plot_value_counts(col_name='userID',df=train_prelim)
users_testset.iloc[:,0] = pd.to_numeric(users_testset.iloc[:,0], errors='coerce').fillna(0, downcast='infer')

# number of items 
items_testset = plot_value_counts(col_name='itemID',df=train_prelim)
items_testset.iloc[:,0] = pd.to_numeric(items_testset.iloc[:,0], errors='coerce').fillna(0, downcast='infer')
                

nb_users = users_testset.shape[0]
nb_items = items_testset.shape[0]


In [ ]:
# ------------------------------ COMPUTING NAIF CLASSIFIORS BASED ON TRAIN SET ----------------------------------


# number of users 
users_testset = plot_value_counts(col_name='userID',df=train_prelim)
users_testset.iloc[:,0] = pd.to_numeric(users_testset.iloc[:,0], errors='coerce').fillna(0, downcast='infer')

# number of items 
items_testset = plot_value_counts(col_name='itemID',df=train_prelim)
items_testset.iloc[:,0] = pd.to_numeric(items_testset.iloc[:,0], errors='coerce').fillna(0, downcast='infer')

nb_users = users_testset.shape[0]
nb_items = items_testset.shape[0]
        
                
              
# classifior based on the transaction number for each item
items_naif1 = np.zeros(items_testset.shape[0])
        
# classifior based on the number of user who did a transaction for each item
items_naif2 = np.zeros(items_testset.shape[0])
        
for i in range (items_testset.shape[0]):
    
            
    df_trans = train_prelim[train_prelim.loc[:,'itemID'] == items_testset.iloc[i,0]]
    items_naif1[i] = np.sum(df_trans.loc[:,'rating'])          
            
            
    for j in range (users_testset.shape[0]): 
        
                
        df_trans2 = df_trans[df_trans.loc[:,'userID'] == users_testset.iloc[j,0]]
        
                    
        if (df_trans2.iloc[0,3] > 0):                 
                    
            items_naif2[i] = items_naif2[i] + 1            
                    
        
                        
# arranger dans le dataframe pandas 'valeurs_items'
items_testset['naif1'] = items_naif1[:]
items_testset['naif2'] = items_naif2[:]
        
        
# order items_testset by items and save in numpy arrays
items_testset = items_testset.sort_values(by='itemID', ascending=False)
npusers_items = np.matrix(items_testset.iloc[:,:])  

In [ ]:
# ---- Evaluer la performance --------------
# --------------------------------------------------------------------------------------------------
        
# init scores 
score = 0 
score_nf1 = 0
score_nf2 = 0


nb_users = users_testset.shape[0]
nb_items = items_testset.shape[0]
        
# number of transactions
divid = nb_users        
      
# ------------------------------------ EVALUATING PERFORMENCE  --------------------------------------

# Create recommendations for all users in the testset

for ind_user in range (nb_users):

    user_id = users_testset.iloc[ind_user,0]
    
                  
    recommendations = recommend(user_id, sparse_user_item, user_vecs, item_vecs, num_items=nb_items) 
    

    # order recommendations by items        
    recommendations = recommendations.sort_values(by='artist', ascending=False)   
            
    # find the "true" rating array of items for user "ind_user"         
    item_matrix = test_prelim2[test_prelim2.loc[:,'userID'] == users_testset.iloc[ind_user,0]]
                    
    sum_mat = np.sum(item_matrix.loc[:,'rating'])   
     
           
    # cases with transactions
    if (sum_mat > 0):     
             
        # order item_matrix by items                
        item_matrix = item_matrix.sort_values(by='itemID', ascending=False)
        npusers_matrix = np.matrix(item_matrix.iloc[:,0::])

        
        # add the "true" rating array of items for user "ind_user" 
        recommendations['rui'] = npusers_matrix[:,3]

        # add the naif classif 1 and 2 
        recommendations['naif1'] = npusers_items[:,3]
        recommendations['naif2'] = npusers_items[:,4]
       
        recommendations = recommendations[recommendations.loc[:,'rui']>(-1)]
        
        recommendations.loc[:,'rui'][recommendations.loc[:,'rui'] > 0] = 1

        # --------------------- Perform on predicted score --------------            
        # order item by predicted score
        
        recommendations = recommendations.sort_values(by='score', ascending=False)

        # calculate penalties 
        penalties = penalt(u_i=recommendations.iloc[:,2])
        score = score + penalties


        # --------------------- Perform on naif 1 --------------            
        # order item by naif 1 array
        recommendations = recommendations.sort_values(by='naif1', ascending=False)
        #print (recommendations.iloc[:,2])

        # calculate penalties 
        penalties = penalt(u_i=recommendations.iloc[:,2])
        score_nf1 = score_nf1 + penalties


        # --------------------- Perform on naif 2 --------------            
        # order item by naif 2 array
        recommendations = recommendations.sort_values(by='naif2', ascending=False)
        #print (recommendations.iloc[:,2])

        # calculate penalties 
        penalties = penalt(u_i=recommendations.iloc[:,2])                
        score_nf2 = score_nf2 + penalties

    # cases without any transaction
    if (sum_mat == 0): 
        

        divid = divid - 1

# compute scores 
print ('scoreTOT')
print (score/divid)
print ('scoreNF1')
print (score_nf1/divid)
print ('scoreNF2')
print (score_nf2/divid)

score = score / divid
score_nf2 = score_nf2 / divid
score_nf1 = score_nf1 / divid


In [ ]:
# représenter les données graphiquement 
d = {"name":['score_manuel','score_naif1','score_naif2','score_collaboratif'],"value":[score_manu,score_nf1,score_nf2,score]
    }

df_score = pd.DataFrame(data=d)

In [ ]:

# Set up the matplotlib figure
f, (ax1) = plt.subplots(1, 1, figsize=(15, 9), sharex=True)

# occurence en valeurs non manquantes
y1 = df_score.iloc[:,1]
g = sns.barplot(x=df_score.iloc[:,0], y=y1, palette='Blues_d', ax=ax1)
ax1.axhline(0, color="k", clip_on=False)

# affiche uniquement un certain nombre de films en légende
plt.locator_params(axis='x', nbins=65)
#ax1.axes.get_xaxis().set_visible(False)
ax1.set_xlabel("numéro d'utilisateur",fontsize=15)
ax1.set_ylabel("nombre de commandes",fontsize=15)
#plt.axis([0, 365, 0, 40])

# Finalize the plot
g = sns.despine(bottom=True)
plt.xticks(rotation=90)
plt.title("nombre de commandes en fonction de l'utilisateur (pour les 1000 premiers utilisateurs)",fontsize=20)
#plt.setp(f.axes, yticks=[])
plt.tight_layout(h_pad=2)
plt.savefig('fig_commandes_utilisateurs.png', dpi=400)